In [1]:
from generalization.utils import get_model
import torch
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10

/mnt/sdd/nguyen.hung.quang/anaconda3/envs/generative/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
device = torch.device('cuda:0')

In [2]:
model = get_model()

23528522


In [24]:
model.load_state_dict(torch.load('log/model_stage_1.pth'))

<All keys matched successfully>

In [28]:
model = torch.load('log/densenet10.pth')

/mnt/sdd/nguyen.hung.quang/anaconda3/envs/generative/lib/python3.10/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'densenet.DenseNet3' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [3]:
sum([p.numel() for p in model.parameters()])

23528522

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((125.3/255, 123.0/255, 113.9/255), (63.0/255, 62.1/255.0, 66.7/255.0)),
])


In [33]:
out_dataset = torchvision.datasets.ImageFolder("data/LSUN", transform=transform)
out_loader = DataLoader(out_dataset, batch_size=64, pin_memory=True)

In [9]:
in_dataset = CIFAR10('../data', train=False, transform=transform)
in_loader = DataLoader(in_dataset, batch_size=64, pin_memory=True)

In [9]:
from detection.odin import get_odin_score

In [10]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

In [48]:

model = model.to(device)

In [49]:
in_score = get_odin_score(model, in_loader, criterion, device=device)

100%|██████████| 157/157 [00:09<00:00, 17.29it/s]


In [50]:
out_score = get_odin_score(model, out_loader, criterion, device=device)

100%|██████████| 157/157 [00:11<00:00, 13.34it/s]


In [51]:
from detection.metrics import *

In [52]:

tpr95(in_score.cpu().numpy(), out_score.cpu().numpy(), 0.1, 0.12)

0.8175249999999998

In [53]:
auroc(in_score.cpu().numpy(), out_score.cpu().numpy(), 0.1, 0.12)

0.9067106500000002

In [54]:
detection_err(in_score.cpu().numpy(), out_score.cpu().numpy(), 0.1, 0.12)

0.11585000000000001

In [12]:
from yaml import Loader
with open('config/jtt.yaml', 'r') as fr:
    config = yaml.load(fr, Loader=Loader)

In [19]:
float(config['stage_1']['lr'])

1e-05

In [25]:
model = model.to(device)
pred, labels = [], []
with torch.no_grad():
    for img, label in in_loader:
        img = img.to(device)
        out = model(img)
        pred.append(out.argmax(dim=1))
        labels.append(label)

In [26]:
pred = torch.cat(pred).cpu()
labels = torch.cat(labels).cpu()

In [27]:
(pred == labels).sum() / len(pred)

tensor(0.8588)